In [1]:
import gzip
import json
import time
import requests 
import pandas as pd
import logging as log
from configparser import RawConfigParser

class Collector():
    
    URL = 'https://neuvoo.com/services/api-new/search?' + \
        'ip=1.1.1.1&useragent=123asd&' + \
        'k=${JOB_TITLE}& ' + \
        'l=${CITY}&f=&o=&p=&r=15&' + \
        'country=${COUNTRY}' + \
        '&format=json&jobdesc=1' + \
        '&start=${START}' + \
        '&limit=${LIMIT}'

    #limit jobs post per resquest (defalt and max = 15)
    LIMIT = 15
    
    #max jobs post per occupation
    MAX = 1000
       
    def __init__(self, args):
        #credentials file
        config = RawConfigParser()
        config.read('./credentials.properties')
     
        self.args = args
        self.username = config.get('ApiSection', 'neuvoo.username')
        self.password = config.get('ApiSection', 'neuvoo.password')
  
        #print(self.username, self.password, self.args['input_file'], self.args['output_file'], self.URL)
    
    def get_job_request(self, job_title, start, limit, location):
        #country, city = location.split(',')
        return requests.get(self.URL
                            .replace("${JOB_TITLE}", job_title)
                            .replace("${START}", start)
                            .replace("${LIMIT}", limit)
                            .replace("${COUNTRY}", 'FR')
                            .replace("${CITY}", location),
                            auth=(self.username, self.password))
    

    def get_jobs(self, job_title, location):
        print("Getting jobs for: " + job_title + " - "+ location)
        data = pd.DataFrame()
        job_titles = [] 
        job_titles_found = []
        companies = []
        descriptions = []
        locations = []
        sources = [] 
        jobkeys = []
        dates = []
        department_name = []
        department_code = []

        for i in range(0, self.MAX, self.LIMIT):
            results = self.get_job_request(job_title, str(i), str(self.LIMIT), location)
            #results_count = len(results.json()['results'])
            totalresults = results.json()['totalresults']
            if totalresults != None:
                for result in results.json()['results']:
                    job_titles.append(job_title)
                    job_titles_found.append(result['jobtitle'])
                    companies.append(result['company'])
                    descriptions.append(result['description'])
                    locations.append(result['formattedLocation'])
                    sources.append(result['source'])
                    jobkeys.append(result['jobkey'])
                    dates.append(result['date'])
                    department_name.append(location)
                    department_code.append(get_department_code(location))
            else:
                break

        data["job_title"] = job_titles
        data["job_title_found"] = job_titles_found
        data["company"] = companies
        data["location"] = locations
        data["sources"] = sources
        data["jobkeys"] = jobkeys
        data["dates"] = dates
        data["description"] = descriptions
        data["department_name"] = department_name
        data["department_code"] = department_code

        return data
    
    def run(self):
        ts = time.gmtime()
        df = pd.DataFrame()
        for location in self.args['locations']:
            df = df.append([self.get_jobs(job_title, location) for job_title in self.args['jobs']])
        
        #tmp_output_file = args['output_file'].replace("${TS}", time.strftime("%Y%m%d%H%M%S", ts))
        #df.to_csv(tmp_output_file, index=False, compression='gzip')

        #print("File saved: ", tmp_output_file)

        return df

In [2]:
departments = pd.read_excel('data/department code.xlsx')
departments = departments[1:]
departments

,NUMÉRO,NOM
1,1,Ain
2,2,Aisne
3,3,Allier
4,4,Alpes-de-Haute-Provence
5,5,Hautes-Alpes
6,6,Alpes-Maritimes
7,7,Ardèche
8,8,Ardennes
9,9,Ariège
10,10,Aube


In [3]:
def get_department_code(name):
    return departments[departments['NOM'].str.startswith(name)]['NUMÉRO'].values[0]
    
get_department_code('Somme')

80

In [4]:
rome_codes = pd.read_csv('data/Sncf codes romes - Sheet1.csv', names=['title', 'rome_code'])
rome_codes.head()

,title,rome_code
0,Agent technique d'unité patrimoine,C1501
1,Assistant technique Patrimoine,C1501
2,Gestionnaire technique immobilier,C1501
3,Assistant de production,C1502
4,Gestionnaire du fichier immobilier,C1502


In [5]:
def get_rome_code(title):
    return rome_codes[rome_codes['title'].str.startswith(title)]['rome_code'].values[0]
    
get_rome_code('Assistant de production')

'C1502'

In [6]:
JOB_TITLES = rome_codes['title'][95:110]
#JOB_TITLES = ['Assistant de production']

LOCATIONS = departments['NOM'].values
#LOCATIONS = ['Paris']

#arguments
args = {
    'name':'neuvoo',
    'jobs': JOB_TITLES,
    'locations': LOCATIONS,
    'output_file': '../../../data/raw/neuvoo/${TS}.csv.gz',
    #...
}

collector = Collector(args)
df = collector.run()
df

Getting jobs for: Chargé Relations Sociales - Ain
Getting jobs for: Contrôleur de gestion RH - Ain
Getting jobs for: Dirigeant Centre Services RH - Ain
Getting jobs for: Dirigeant d'activité RH - Ain
Getting jobs for: Gestionnaire utilisation RH - Ain
Getting jobs for: Responsable du pole Ressources Humaines - Ain
Getting jobs for: Assistant de gestion - Ain
Getting jobs for: Assistant en gestion de site - Ain
Getting jobs for: Secrétaire assistant - Ain
Getting jobs for: Chargé de Relation Client - Ain
Getting jobs for: Responsable de service clientèle - Ain
Getting jobs for: Service Manager - Ain
Getting jobs for: Consultant Sécurité SI - Ain
Getting jobs for: Chargé d'études Ingénierie Télécommunications - Ain
Getting jobs for: Dirigeant Agence Services Télécoms Informatique - Ain
Getting jobs for: Chargé Relations Sociales - Aisne
Getting jobs for: Contrôleur de gestion RH - Aisne
Getting jobs for: Dirigeant Centre Services RH - Aisne
Getting jobs for: Dirigeant d'activité RH - Ais

Getting jobs for: Gestionnaire utilisation RH - Aube
Getting jobs for: Responsable du pole Ressources Humaines - Aube
Getting jobs for: Assistant de gestion - Aube
Getting jobs for: Assistant en gestion de site - Aube
Getting jobs for: Secrétaire assistant - Aube
Getting jobs for: Chargé de Relation Client - Aube
Getting jobs for: Responsable de service clientèle - Aube
Getting jobs for: Service Manager - Aube
Getting jobs for: Consultant Sécurité SI - Aube
Getting jobs for: Chargé d'études Ingénierie Télécommunications - Aube
Getting jobs for: Dirigeant Agence Services Télécoms Informatique - Aube
Getting jobs for: Chargé Relations Sociales - Aude
Getting jobs for: Contrôleur de gestion RH - Aude
Getting jobs for: Dirigeant Centre Services RH - Aude
Getting jobs for: Dirigeant d'activité RH - Aude
Getting jobs for: Gestionnaire utilisation RH - Aude
Getting jobs for: Responsable du pole Ressources Humaines - Aude
Getting jobs for: Assistant de gestion - Aude
Getting jobs for: Assistan

Getting jobs for: Responsable de service clientèle - Corrèze
Getting jobs for: Service Manager - Corrèze
Getting jobs for: Consultant Sécurité SI - Corrèze
Getting jobs for: Chargé d'études Ingénierie Télécommunications - Corrèze
Getting jobs for: Dirigeant Agence Services Télécoms Informatique - Corrèze
Getting jobs for: Chargé Relations Sociales - Corse-du-Sud
Getting jobs for: Contrôleur de gestion RH - Corse-du-Sud
Getting jobs for: Dirigeant Centre Services RH - Corse-du-Sud
Getting jobs for: Dirigeant d'activité RH - Corse-du-Sud
Getting jobs for: Gestionnaire utilisation RH - Corse-du-Sud
Getting jobs for: Responsable du pole Ressources Humaines - Corse-du-Sud
Getting jobs for: Assistant de gestion - Corse-du-Sud
Getting jobs for: Assistant en gestion de site - Corse-du-Sud
Getting jobs for: Secrétaire assistant - Corse-du-Sud
Getting jobs for: Chargé de Relation Client - Corse-du-Sud
Getting jobs for: Responsable de service clientèle - Corse-du-Sud
Getting jobs for: Service Man

Getting jobs for: Contrôleur de gestion RH - Eure-et-Loir
Getting jobs for: Dirigeant Centre Services RH - Eure-et-Loir
Getting jobs for: Dirigeant d'activité RH - Eure-et-Loir
Getting jobs for: Gestionnaire utilisation RH - Eure-et-Loir
Getting jobs for: Responsable du pole Ressources Humaines - Eure-et-Loir
Getting jobs for: Assistant de gestion - Eure-et-Loir
Getting jobs for: Assistant en gestion de site - Eure-et-Loir
Getting jobs for: Secrétaire assistant - Eure-et-Loir
Getting jobs for: Chargé de Relation Client - Eure-et-Loir
Getting jobs for: Responsable de service clientèle - Eure-et-Loir
Getting jobs for: Service Manager - Eure-et-Loir
Getting jobs for: Consultant Sécurité SI - Eure-et-Loir
Getting jobs for: Chargé d'études Ingénierie Télécommunications - Eure-et-Loir
Getting jobs for: Dirigeant Agence Services Télécoms Informatique - Eure-et-Loir
Getting jobs for: Chargé Relations Sociales - Finistère
Getting jobs for: Contrôleur de gestion RH - Finistère
Getting jobs for: 

Getting jobs for: Assistant de gestion - Indre-et-Loire
Getting jobs for: Assistant en gestion de site - Indre-et-Loire
Getting jobs for: Secrétaire assistant - Indre-et-Loire
Getting jobs for: Chargé de Relation Client - Indre-et-Loire
Getting jobs for: Responsable de service clientèle - Indre-et-Loire
Getting jobs for: Service Manager - Indre-et-Loire
Getting jobs for: Consultant Sécurité SI - Indre-et-Loire
Getting jobs for: Chargé d'études Ingénierie Télécommunications - Indre-et-Loire
Getting jobs for: Dirigeant Agence Services Télécoms Informatique - Indre-et-Loire
Getting jobs for: Chargé Relations Sociales - Isère
Getting jobs for: Contrôleur de gestion RH - Isère
Getting jobs for: Dirigeant Centre Services RH - Isère
Getting jobs for: Dirigeant d'activité RH - Isère
Getting jobs for: Gestionnaire utilisation RH - Isère
Getting jobs for: Responsable du pole Ressources Humaines - Isère
Getting jobs for: Assistant de gestion - Isère
Getting jobs for: Assistant en gestion de site 

...

Getting jobs for: Assistant de gestion - Hautes-Pyrénées
Getting jobs for: Assistant en gestion de site - Hautes-Pyrénées
Getting jobs for: Secrétaire assistant - Hautes-Pyrénées
Getting jobs for: Chargé de Relation Client - Hautes-Pyrénées
Getting jobs for: Responsable de service clientèle - Hautes-Pyrénées
Getting jobs for: Service Manager - Hautes-Pyrénées
Getting jobs for: Consultant Sécurité SI - Hautes-Pyrénées
Getting jobs for: Chargé d'études Ingénierie Télécommunications - Hautes-Pyrénées
Getting jobs for: Dirigeant Agence Services Télécoms Informatique - Hautes-Pyrénées
Getting jobs for: Chargé Relations Sociales - Pyrénées-Orientales
Getting jobs for: Contrôleur de gestion RH - Pyrénées-Orientales
Getting jobs for: Dirigeant Centre Services RH - Pyrénées-Orientales
Getting jobs for: Dirigeant d'activité RH - Pyrénées-Orientales
Getting jobs for: Gestionnaire utilisation RH - Pyrénées-Orientales
Getting jobs for: Responsable du pole Ressources Humaines - Pyrénées-Orientales
G

Getting jobs for: Secrétaire assistant - Haute-Savoie
Getting jobs for: Chargé de Relation Client - Haute-Savoie
Getting jobs for: Responsable de service clientèle - Haute-Savoie
Getting jobs for: Service Manager - Haute-Savoie
Getting jobs for: Consultant Sécurité SI - Haute-Savoie
Getting jobs for: Chargé d'études Ingénierie Télécommunications - Haute-Savoie
Getting jobs for: Dirigeant Agence Services Télécoms Informatique - Haute-Savoie
Getting jobs for: Chargé Relations Sociales - Paris
Getting jobs for: Contrôleur de gestion RH - Paris
Getting jobs for: Dirigeant Centre Services RH - Paris
Getting jobs for: Dirigeant d'activité RH - Paris
Getting jobs for: Gestionnaire utilisation RH - Paris
Getting jobs for: Responsable du pole Ressources Humaines - Paris
Getting jobs for: Assistant de gestion - Paris
Getting jobs for: Assistant en gestion de site - Paris
Getting jobs for: Secrétaire assistant - Paris
Getting jobs for: Chargé de Relation Client - Paris
Getting jobs for: Responsab

Getting jobs for: Chargé d'études Ingénierie Télécommunications - Var
Getting jobs for: Dirigeant Agence Services Télécoms Informatique - Var
Getting jobs for: Chargé Relations Sociales - Vaucluse
Getting jobs for: Contrôleur de gestion RH - Vaucluse
Getting jobs for: Dirigeant Centre Services RH - Vaucluse
Getting jobs for: Dirigeant d'activité RH - Vaucluse
Getting jobs for: Gestionnaire utilisation RH - Vaucluse
Getting jobs for: Responsable du pole Ressources Humaines - Vaucluse
Getting jobs for: Assistant de gestion - Vaucluse
Getting jobs for: Assistant en gestion de site - Vaucluse
Getting jobs for: Secrétaire assistant - Vaucluse
Getting jobs for: Chargé de Relation Client - Vaucluse
Getting jobs for: Responsable de service clientèle - Vaucluse
Getting jobs for: Service Manager - Vaucluse
Getting jobs for: Consultant Sécurité SI - Vaucluse
Getting jobs for: Chargé d'études Ingénierie Télécommunications - Vaucluse
Getting jobs for: Dirigeant Agence Services Télécoms Informatique

Getting jobs for: Contrôleur de gestion RH - Seine-St-Denis
Getting jobs for: Dirigeant Centre Services RH - Seine-St-Denis
Getting jobs for: Dirigeant d'activité RH - Seine-St-Denis
Getting jobs for: Gestionnaire utilisation RH - Seine-St-Denis
Getting jobs for: Responsable du pole Ressources Humaines - Seine-St-Denis
Getting jobs for: Assistant de gestion - Seine-St-Denis
Getting jobs for: Assistant en gestion de site - Seine-St-Denis
Getting jobs for: Secrétaire assistant - Seine-St-Denis
Getting jobs for: Chargé de Relation Client - Seine-St-Denis
Getting jobs for: Responsable de service clientèle - Seine-St-Denis
Getting jobs for: Service Manager - Seine-St-Denis
Getting jobs for: Consultant Sécurité SI - Seine-St-Denis
Getting jobs for: Chargé d'études Ingénierie Télécommunications - Seine-St-Denis
Getting jobs for: Dirigeant Agence Services Télécoms Informatique - Seine-St-Denis
Getting jobs for: Chargé Relations Sociales - Val-de-Marne
Getting jobs for: Contrôleur de gestion RH

,job_title,job_title_found,company,location,sources,jobkeys,dates,description,department_name,department_code
0,Assistant de gestion,Assistant(te) Gestionnaire de biens immobilier...,SEMCODA,"Bourg-en-Bresse, Auvergne-Rhône-Alpes, FR",Taleez,87472b74fcae,2020-07-16,CARRE PRO Immobilier est la marque de l’immobi...,Ain,1.0
0,Chargé de Relation Client,CHARGÉ DE RELATION CLIENT BILINGUE ESPAGNOL H/F,Ras,"Thil, Auvergne-Rhône-Alpes, FR",Monster,da6703e97de4,2020-07-15,Votre agence RAS Lyon OVERSEAS recherche pour ...,Ain,1.0
0,Service Manager,MANAGER CAISSES SERVICE CLIENT -,Commerce RH,"Mionnay, Auvergne-Rhône-Alpes, FR",Commerce RH,d4a8821a6e3a,2020-07-16,Commerce/Distribution:/Restauration Niveau d'é...,Ain,1.0
0,Service Manager,MANAGER STATION DE SERVICE,Commerce RH,"Roumoules, Provence-Alpes-Côte d'Azur, FR",Commerce RH,55546452a9dd,2020-07-16,Distribution Niveau d'étudesBac+1/2 Niveau d'e...,Alpes-de-Haute-Provence,4.0
0,Assistant de gestion,Assistant de gestion en cabinet comptable (h/f),TERCIO,"null, Provence-Alpes-Côte d'Azur, FR",Talentplug,9fba826751b3,2020-07-13,A propos de l'entreprise :TERCIO et son bureau...,Hautes-Alpes,5.0
0,Secrétaire assistant,Assistant dentaire ou secrÉtaire mÉdical h/f,Pôle emploi,"Briançon, Provence-Alpes-Côte d'Azur, FR",Jobvitae,63d491687d50,2020-07-17,Nous recherchons un(e) Assistant dentaire ou s...,Hautes-Alpes,5.0
0,Service Manager,MANAGER STATION DE SERVICE,Commerce RH,"Briançon, Provence-Alpes-Côte d'Azur, FR",Commerce RH,d01c38899e66,2020-07-16,Distribution Niveau d'étudesBac+1/2 Niveau d'e...,Hautes-Alpes,5.0
0,Assistant de gestion,Assistant(e) de gestion h/f,Page Personnel,"Nice, Provence-Alpes-Côte d'Azur, FR",Keljob,bdfe21d0801b,2020-07-19,<h2>Le poste</h2>En tant qu'Assistant de Gesti...,Alpes-Maritimes,6.0
1,Assistant de gestion,Assistant de gestion en cabinet comptable (h/f),TERCIO,"Nice, Provence-Alpes-Côte d'Azur, FR",DLSI,a033f0e43d7c,2020-07-14,TERCIO et son bureau de Nice recrute pour son ...,Alpes-Maritimes,6.0
2,Assistant de gestion,Assistant(e) de Gestion/Direction • Back Offic...,MARINESCENCE,"Cannes, Provence-Alpes-Côte d'Azur, FR",Jobsoid,efeb4bd69f92,2020-07-13,A propos de l'entreprise\rC'est une entreprise...,Alpes-Maritimes,6.0


In [7]:
df['rome_code'] = [get_rome_code(title) for title in df['job_title']]
df.head()

,job_title,job_title_found,company,location,sources,jobkeys,dates,description,department_name,department_code,rome_code
0,Assistant de gestion,Assistant(te) Gestionnaire de biens immobilier...,SEMCODA,"Bourg-en-Bresse, Auvergne-Rhône-Alpes, FR",Taleez,87472b74fcae,2020-07-16,CARRE PRO Immobilier est la marque de l’immobi...,Ain,1.0,M1605
0,Chargé de Relation Client,CHARGÉ DE RELATION CLIENT BILINGUE ESPAGNOL H/F,Ras,"Thil, Auvergne-Rhône-Alpes, FR",Monster,da6703e97de4,2020-07-15,Votre agence RAS Lyon OVERSEAS recherche pour ...,Ain,1.0,M1704
0,Service Manager,MANAGER CAISSES SERVICE CLIENT -,Commerce RH,"Mionnay, Auvergne-Rhône-Alpes, FR",Commerce RH,d4a8821a6e3a,2020-07-16,Commerce/Distribution:/Restauration Niveau d'é...,Ain,1.0,M1704
0,Service Manager,MANAGER STATION DE SERVICE,Commerce RH,"Roumoules, Provence-Alpes-Côte d'Azur, FR",Commerce RH,55546452a9dd,2020-07-16,Distribution Niveau d'étudesBac+1/2 Niveau d'e...,Alpes-de-Haute-Provence,4.0,M1704
0,Assistant de gestion,Assistant de gestion en cabinet comptable (h/f),TERCIO,"null, Provence-Alpes-Côte d'Azur, FR",Talentplug,9fba826751b3,2020-07-13,A propos de l'entreprise :TERCIO et son bureau...,Hautes-Alpes,5.0,M1605


In [8]:
df.to_clipboard(index=False, header=False)

In [15]:
df.loc[df.job_title == 'promoteur des ventes', 'rome_code'] = 'M1706'
df.loc[df.job_title == 'responsable de secteur', 'rome_code'] = 'D1509'

df

C:\Users\neobrain\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,job_title,job_title_found,company,location,sources,jobkeys,dates,description,department_name,department_code,rome_code


In [44]:
#Status Code, Content Type, Enconding, Total Results
#r.status_code, r.headers['content-type'], r.encoding, r.json()['totalresults']

result = collector.get_job_request('Data scientist', '0', '15', "us,Silicon Valley")
assert result.status_code == 200
result.json()['results']

In [46]:
collector.get_job_request('ventes', '0', '15', "FR, Paris").json()

{'totalresults': 211,
 'results': [{'jobkey': 'ce45ab23fa3b',
   'jobtitle': 'Chef de ventes Véhicules Neufs H/F Région parisienne',
   'company': 'RENAULT RETAIL GROUP',
   'city': 'Paris',
   'state': 'Île-de-France',
   'country': 'FR',
   'formattedLocation': 'Paris, Île-de-France, FR',
   'source': 'RENAULT RETAIL GROUP',
   'date': '2020-07-02',
   'description': "Des métiers « passion », des produits « plaisir », des possibilités de s'épanouir professionnellement dans un environnement varié et dynamique centré sur nos clients et leur satisfaction…Renault Retail Group est une filiale à 100% du constructeur Renault. Le monde de l'automobile est en perpétuelle évolution et nous nous attachons à la conception de véhicules innovants, séduisants et encore plus respectueux de l'environnement. RRG repose sur nos valeurs telles que : garder toujours le client à l'esprit, apporter sa contribution, parler vrai, apprendre au quotidien et faire simple. Notre culture d'entreprise est notre fo

In [47]:
collector.get_job_request('promotion des ventes', '0', '15', "FR, Paris").json()

{'totalresults': None, 'results': None}